In [1]:
from itertools import compress

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import Crippen
from rdkit.Chem import rdchem

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Some auxiliary functions

In [4]:
def evaluate_performance(predicted, true):
    '''Calculate basic performance characteristics: MAE, RMSE and R^2'''
    model_mse = mean_squared_error(true, predicted)
    model_rmse = np.sqrt(model_mse)
    print ('----> RMSE= \t\t{}'.format(model_rmse))
    
    model_mae = mean_absolute_error(true, predicted)
    print ('----> MAE= \t\t{}'.format(model_mae))
    
        
    model_r_square = r2_score(true, predicted)
    print ('----> R^2= \t\t{}'.format(model_r_square))

In [5]:
def descriptor_calculator(mol_array, module):
    '''Output list with calculated descriptor '''
    result=[module(x) for x in mol_array]
    return result

def add_desc(df, mol_array, d_dictionary):
    '''Add descriptor columns to pandas df'''
    for key, value in d_dictionary.items():
        print('calculating: \t', key)
        df[key] = descriptor_calculator(mol_array, value)
    return df


'''
Dictionary with all descroptors
Comment out to exclude
http://www.rdkit.org/Python_Docs/rdkit.Chem.rdMolDescriptors-module.html#CalcExactMolWt
'''
descriptor_dict={#'Crippen_SlogP': Crippen._pyMolLogP, # standard one
                 'Crippen_MolLogP': Crippen.MolLogP, # 
                 'Crippen_MolMR': Crippen.MolMR,
                 'Chi0n': AllChem.CalcChi0n,
                 'Chi0v': AllChem.CalcChi0v,
                 'Chi1n': AllChem.CalcChi1n,
                 'Chi1v': AllChem.CalcChi1v,
                 'Chi2n': AllChem.CalcChi2n,
                 'Chi2v': AllChem.CalcChi2v,
                 'Chi3n': AllChem.CalcChi3n,
                 'Chi3v': AllChem.CalcChi3v,
                 'Chi4n': AllChem.CalcChi4n,
                 'Chi4v': AllChem.CalcChi4v,
                 #'ChiNn': AllChem.CalcChiNn,
                 #'ChiNv': AllChem.CalcChiNv,
                 #'Eccentricity': AllChem.CalcEccentricity,
                 'ExactMolWt': AllChem.CalcExactMolWt,
                 'FractionCSP3': AllChem.CalcFractionCSP3,
                 'allKierAlpha': AllChem.CalcHallKierAlpha,
                 #'InertialShapeFactor': AllChem.CalcInertialShapeFactor,
                 'Kappa1': AllChem.CalcKappa1,
                 'Kappa2': AllChem.CalcKappa2,
                 'Kappa3': AllChem.CalcKappa3,
                 'LabuteASA': AllChem.CalcLabuteASA,
                 'MolFormula': AllChem.CalcMolFormula,
                 #'NPR1': AllChem.CalcNPR1,
                 #'NPR2': AllChem.CalcNPR2,
                 'N_AliphCarbC': AllChem.CalcNumAliphaticCarbocycles,
                 'N_AliphHetC': AllChem.CalcNumAliphaticHeterocycles,
                 'N_AliphRing': AllChem.CalcNumAliphaticRings,
                 'N_AmideBonds': AllChem.CalcNumAmideBonds,
                 'N_AromCarbC': AllChem.CalcNumAromaticCarbocycles,
                 'N_AromHetC': AllChem.CalcNumAromaticHeterocycles,
                 'N_AromRing': AllChem.CalcNumAromaticRings,
                 'N_AtomStereoCenters': AllChem.CalcNumAtomStereoCenters,
                 'N_BridgeheadAtoms': AllChem.CalcNumBridgeheadAtoms,
                 'N_HBA': AllChem.CalcNumHBA,
                 'N_HBD': AllChem.CalcNumHBD,
                 'N_Heteroatoms': AllChem.CalcNumHeteroatoms,
                 'N_Heterocycles': AllChem.CalcNumHeterocycles,
                 'N_LipinskiHBA': AllChem.CalcNumLipinskiHBA,
                 'N_LipinskiHBD': AllChem.CalcNumLipinskiHBD,
                 'N_Rings': AllChem.CalcNumRings,
                 'N_RotatableBonds': AllChem.CalcNumRotatableBonds,
                 'N_CarbC': AllChem.CalcNumSaturatedCarbocycles,
                 'N_SatHetC': AllChem.CalcNumSaturatedHeterocycles,
                 'N_SatRing': AllChem.CalcNumSaturatedRings,
                 'N_SpiroAt': AllChem.CalcNumSpiroAtoms,
                 'N_UnsAtSterCent': AllChem.CalcNumUnspecifiedAtomStereoCenters}
                 #'PBF': AllChem.CalcPBF,
                 #'PMI1': AllChem.CalcPMI1,
                 #'PMI2': AllChem.CalcPMI2,
                 #'PMI3': AllChem.CalcPMI3,
                 #'RadiusOfGyration': AllChem.CalcRadiusOfGyration,
                 #'SpherInd': AllChem.CalcSpherocityIndex,
                 #'AtPairAtCode': AllChem.GetAtomPairAtomCode}
                 #'TPSA': AllChem.CalcTPSA}

In [6]:
# total descriptors to be calculated:
len(descriptor_dict)

42

## Reading the .xslx from Lombardo et al.

In [7]:
pk_data = pd.read_excel('Supplemental_82966_revised_corrected.xlsx', sheet_name='Data_sheet', skiprows= lambda x: x in [0, 6], header= 6) 

In [8]:
pk_data.head()

,Name,CAS #,SMILES,human VDss (L/kg),human CL (mL/min/kg),fraction unbound \nin plasma (fu),MRT (h),terminal t1/2 (h),Reference,Comments,Notes,Year of first disclosure,MW,HBA,HBD,TPSA_NO,RotBondCount,moka_ionState7.4,MoKa.LogP,MoKa.LogD7.4
0,α-hANP,85637-73-6,N1[C@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C(...,0.20,25.40,NaN,0.13,0.22,"Nakao, K.; Sugawara, A.; Morii, N.; Sakamoto, ...",α-Human Atrial Natriuretic Peptide. MRT calcul...,NaN,1982,3080.44,84,53,1403.4,75,cationic,-9.00,-9.00
1,(-)dOTC,160707-69-7,c1cn(c(=O)nc1N)[C@H]2CO[C@H](S2)CO,1.18,3.00,NaN,6.50,19.20,"PATRICK F. SMITH, ALAN FORREST, CHARLES H. BAL...",Dosed as 100 mg of racemate. (-) form called a...,NaN,1994,229.26,6,2,90.4,2,neutral,-1.90,-1.90
2,(+)dOTC,160707-68-6,c1cn(c(=O)nc1N)[C@@H]2CO[C@@H](S2)CO,0.84,3.90,NaN,3.60,8.92,"PATRICK F. SMITH, ALAN FORREST, CHARLES H. BAL...",Dosed as 100 mg of racemate. (-) form called a...,NaN,1994,229.26,6,2,90.4,2,neutral,-1.90,-1.90
3,"1,3-DCQA",19870-46-3,O[C@@H]1C[C@@](C[C@@H](OC(=O)\C=C\c2ccc(O)c(O)...,0.79,8.70,NaN,1.50,1.37,"Wen-Zheng Ju,Yang Zhao,Fang Liu,Ting Wua,Jun Z...","1,3-dicaffeoylquinic acid. Data digitized from...",NaN,1964,516.45,12,7,211.3,9,anionic,2.10,-1.60
4,16-acetyl gitoxin,7242-07-1,C1([C@@H]2[C@@]3([C@@]([C@@H]4[C@@H]([C@@]5(CC...,0.78,0.18,0.07,72.30,51.60,"Hausten, K.-O. On the Pharmacokinetics of 16-A...","16-O-acetylgitoxin. N=6, 65.3 kg average weigh...",NaN,1960,822.98,15,5,209.1,9,neutral,0.93,0.93


## Checking original source data

In [9]:
pk_data.shape

(1352, 20)

In [10]:
pk_data['human VDss (L/kg)'].describe()

count    1315.000000
mean        3.829262
std        20.787797
min         0.030000
25%         0.300000
50%         0.910000
75%         2.600000
max       700.000000
Name: human VDss (L/kg), dtype: float64

## Generating RDKit's mols

In [11]:
PandasTools.AddMoleculeColumnToFrame(pk_data, smilesCol='SMILES', molCol='Mol')

[18:28:14] Explicit valence for atom # 3 O, 3, is greater than permitted
RDKit ERROR: [18:28:14] Explicit valence for atom # 3 O, 3, is greater than permitted
[18:28:14] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [18:28:14] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [18:28:14] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit ERROR: [18:28:14] Explicit valence for atom # 3 O, 3, is greater than permitted
[18:28:14] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit ERROR: [18:28:14] SMILES Parse Error: syntax error while parsing: C(C([N+]([Gd+++]([N+]1(C2)C3)([N+](C2)(C2)C4)([O-]5)([O-]6)([O-]C3=O)([O-]C4=O)[O-]C2=O)(CC6=O)CC5=O)C1)C(=CC=C1OCC)C=C1
RDKit ERROR: [18:28:14] SMILES Parse Error: Failed parsing SMILES 'C(C([N+]([Gd+++]([N+]1(C2)C3)([N+](C2)(C2)C4)([O-]5)([O-]6)([O-]C3=O)([O-]C4=O)[O-]C2=O)(CC6=O)CC5=O)C1)C(=CC=C1OCC)C=C1' for input: 'C(C([N+]([Gd+++]([N+]1(C2)C3)([N+](C2)(C2)C

## --> throws some errors

## => filtering the compounds which RDKit can't process

In [12]:
pk_data= pk_data[pk_data['Mol'].apply(lambda x: isinstance(x, rdchem.Mol))]

## Checking the size (1352 records initially)

In [13]:
pk_data.shape

(1340, 21)

## Calculating descriptors

In [14]:
pk_data = add_desc(pk_data.copy(), pk_data['Mol'], descriptor_dict)

calculating: 	 Crippen_MolLogP
calculating: 	 Crippen_MolMR
calculating: 	 Chi0n
calculating: 	 Chi0v
calculating: 	 Chi1n
calculating: 	 Chi1v
calculating: 	 Chi2n
calculating: 	 Chi2v
calculating: 	 Chi3n
calculating: 	 Chi3v
calculating: 	 Chi4n
calculating: 	 Chi4v
calculating: 	 ExactMolWt
calculating: 	 FractionCSP3
calculating: 	 allKierAlpha
calculating: 	 Kappa1
calculating: 	 Kappa2
calculating: 	 Kappa3
calculating: 	 LabuteASA
calculating: 	 MolFormula
calculating: 	 N_AliphCarbC
calculating: 	 N_AliphHetC
calculating: 	 N_AliphRing
calculating: 	 N_AmideBonds
calculating: 	 N_AromCarbC
calculating: 	 N_AromHetC
calculating: 	 N_AromRing
calculating: 	 N_AtomStereoCenters
calculating: 	 N_BridgeheadAtoms
calculating: 	 N_HBA
calculating: 	 N_HBD
calculating: 	 N_Heteroatoms
calculating: 	 N_Heterocycles
calculating: 	 N_LipinskiHBA
calculating: 	 N_LipinskiHBD
calculating: 	 N_Rings
calculating: 	 N_RotatableBonds
calculating: 	 N_CarbC
calculating: 	 N_SatHetC
calculating:

## Filtering out records without 'MoKa.LogP' in the .xsls

In [15]:
pk_data = pk_data[pk_data['MoKa.LogP'].notna()]

In [16]:
pk_data.shape

(1337, 63)

## Lipinksi filter

In [17]:
pk_data= pk_data[(pk_data['HBD'] <= 5) & (pk_data['HBA'] <= 10) & (pk_data['ExactMolWt'] <= 500) & (pk_data['MoKa.LogP'] <= 5)]

In [18]:
pk_data.shape

(921, 63)

## Selecting relevant RDKit descriptors for evaluation
## ..and the target variables

In [19]:
features = ['MW', 'HBA', 'HBD', 'TPSA_NO', 'RotBondCount',
       'MoKa.LogP', 'MoKa.LogD7.4', 'Crippen_MolMR',
       'Chi0n', 'Chi0v', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v',
       'Chi4n', 'Chi4v', 'ExactMolWt', 'FractionCSP3', 'allKierAlpha',
       'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'N_AliphCarbC',
       'N_AliphHetC', 'N_AliphRing', 'N_AmideBonds', 'N_AromCarbC',
       'N_AromHetC', 'N_AromRing', 'N_AtomStereoCenters', 'N_BridgeheadAtoms',
       'N_HBA', 'N_HBD', 'N_Heteroatoms', 'N_Heterocycles', 'N_LipinskiHBA',
       'N_LipinskiHBD', 'N_Rings', 'N_RotatableBonds', 'N_CarbC', 'N_SatHetC',
       'N_SatRing', 'N_SpiroAt', 'N_UnsAtSterCent']

target_vss = 'human VDss (L/kg)'
target_cl = 'human CL (mL/min/kg)'

target_log_vss = 'log_vss'
target_log_cl = 'log_cl'

In [20]:
len(features) # 47 descriptors for evaluation

47

## Selecting data for separate modelling of Vss and PK

## ..and looking at the size of the corresponding data subsets

In [21]:
pk_vss = pk_data[pk_data['human VDss (L/kg)'].notna()]
pk_cl = pk_data[pk_data['human CL (mL/min/kg)'].notna()]

print(pk_data.shape)
print(pk_vss.shape)
print(pk_cl.shape)

(921, 63)
(896, 63)
(919, 63)


## Visual inspection

In [22]:
# Clearance raw

hist, edges = pd.np.histogram(pk_cl['human CL (mL/min/kg)'], density=False, bins=50)

p = figure()
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")

p.xaxis.axis_label = 'human CL (mL/min/kg)'
p.yaxis.axis_label = 'Value counts'

p.xaxis.axis_label_text_font_size = '20pt'
p.yaxis.axis_label_text_font_size = '20pt'

show(p)

In [23]:
# same for Vss

hist, edges = pd.np.histogram(pk_vss['human VDss (L/kg)'], density=False, bins=50)

p = figure()
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")

p.xaxis.axis_label = 'human VDss (L/kg)'
p.yaxis.axis_label = 'Value counts'

p.xaxis.axis_label_text_font_size = '20pt'
p.yaxis.axis_label_text_font_size = '20pt'

show(p)

## Does not look good
## -> let's use log values

In [24]:
pk_vss['log_vss'] = np.log(pk_vss['human VDss (L/kg)'])
pk_cl['log_cl'] = np.log(pk_cl['human CL (mL/min/kg)'])

In [25]:
# log(clearance)

hist, edges = pd.np.histogram(pk_cl['log_cl'], density=False, bins=50)

p = figure()
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")

p.xaxis.axis_label = 'log(human CL (mL/min/kg))'
p.yaxis.axis_label = 'Value counts'

p.xaxis.axis_label_text_font_size = '20pt'
p.yaxis.axis_label_text_font_size = '20pt'

show(p)

In [26]:
# log(Vss)

hist, edges = pd.np.histogram(pk_vss['log_vss'], density=False, bins=50)

p = figure()
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")

p.xaxis.axis_label = 'log(human VDss (L/kg))'
p.yaxis.axis_label = 'Value counts'

p.xaxis.axis_label_text_font_size = '20pt'
p.yaxis.axis_label_text_font_size = '20pt'

show(p)

## Looks better;

## Splitting into train- and test-sets

In [27]:
pk_vss_train, pk_vss_test = train_test_split(pk_vss, test_size=0.2, random_state=4)
pk_cl_train, pk_cl_test = train_test_split(pk_cl, test_size=0.2, random_state=4)

In [28]:
# checking last time before modelling
pk_vss_train[features]

,MW,HBA,HBD,TPSA_NO,RotBondCount,MoKa.LogP,MoKa.LogD7.4,Crippen_MolMR,Chi0n,Chi0v,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,ExactMolWt,FractionCSP3,allKierAlpha,Kappa1,Kappa2,Kappa3,LabuteASA,N_AliphCarbC,N_AliphHetC,N_AliphRing,N_AmideBonds,N_AromCarbC,N_AromHetC,N_AromRing,N_AtomStereoCenters,N_BridgeheadAtoms,N_HBA,N_HBD,N_Heteroatoms,N_Heterocycles,N_LipinskiHBA,N_LipinskiHBD,N_Rings,N_RotatableBonds,N_CarbC,N_SatHetC,N_SatRing,N_SpiroAt,N_UnsAtSterCent
615,482.53,8,3,89.7,2,2.50,-0.29,136.9037,20.201696,20.201696,12.381564,12.381564,10.265916,10.265916,8.646890,8.646890,7.378615,7.378615,482.195405,0.321429,-3.43,20.731696,6.657457,2.147352,205.910823,0,3,3,1,3,2,5,5,2,7,3,8,5,8,3,8,2,0,1,1,0,1
352,228.21,8,3,120.7,2,-2.00,-2.00,52.3240,8.353529,8.353529,4.765733,4.765733,3.519225,3.519225,2.439845,2.439845,1.603095,1.603095,228.085855,0.625000,-1.51,10.989791,4.160619,1.896064,90.322623,0,1,1,0,0,1,1,3,0,8,3,8,2,8,4,2,2,0,1,1,0,0
630,261.09,4,1,41.6,5,0.88,0.88,59.1912,7.916600,10.322885,4.597962,7.244517,2.946928,5.883787,1.947937,4.606453,1.318848,3.860557,260.024820,1.000000,0.69,12.758074,6.311144,3.368384,94.441535,0,1,1,0,0,0,0,1,0,2,1,7,1,4,1,1,5,0,1,1,0,1
644,281.31,4,1,57.6,3,2.40,-0.59,79.4483,11.614183,11.614183,6.802268,6.802268,5.199902,5.199902,3.896570,3.896570,2.548428,2.548428,281.105193,0.176471,-2.62,13.367084,5.052450,2.288972,122.184488,0,1,1,1,2,0,2,1,0,2,1,4,1,4,1,3,3,0,0,0,0,1
568,389.38,9,2,123.0,5,1.70,-0.26,100.3927,15.261063,15.261063,8.978042,8.978042,7.058810,7.058810,5.167537,5.167537,3.595955,3.595955,389.149932,0.444444,-3.01,18.357814,6.860777,2.988630,158.745538,1,1,2,0,0,2,2,1,0,8,2,10,3,9,3,4,5,1,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,323.41,6,2,78.5,3,1.70,0.20,82.4362,12.579842,13.396339,7.589462,9.072625,6.215525,7.897927,4.544292,5.891185,3.391138,4.293093,323.130363,0.533333,-1.60,15.301626,5.887683,3.250242,130.456212,1,1,2,2,1,0,1,2,0,4,2,7,1,6,2,3,3,1,1,2,0,0
1067,180.25,3,1,33.6,4,0.91,-2.20,50.6257,7.830153,7.830153,5.370969,5.370969,4.380316,4.380316,3.135372,3.135372,2.096324,2.096324,180.126263,0.900000,-0.73,7.653224,2.893059,1.644159,78.564966,2,1,3,0,0,0,0,0,0,3,1,3,1,3,1,3,3,2,0,2,0,0
698,229.26,6,2,88.1,2,-1.00,-1.00,56.3632,7.867351,8.683847,4.430532,5.479287,3.078450,4.270900,2.044409,3.153980,1.330063,2.260751,229.052112,0.500000,-1.05,10.467220,4.213447,2.137866,90.066910,0,1,1,0,0,1,1,2,0,7,2,7,2,6,3,2,2,0,1,1,0,0
292,324.39,3,0,36.3,6,4.10,2.00,90.9140,14.050519,14.050519,8.132551,8.132551,6.513142,6.513142,4.588190,4.588190,3.408709,3.408709,324.163792,0.350000,-2.22,16.631264,6.721078,3.055802,141.833697,0,1,1,0,2,0,2,1,0,3,0,4,1,3,0,3,5,0,0,0,0,1


In [29]:
# are there any errors to expect? (with NaN Vss records)

pk_vss_train[features].isnull().values.any()

False

# Evaluating linear model (Vss)

## (alpha and iteration number selected by previous manual selection of 6-7 different options)

# 1. raw data

In [30]:
lasso_raw = linear_model.Lasso(alpha=0.0001, max_iter=10_000_000)
lasso_raw.fit(pk_vss_train[features], pk_vss_train[target_vss])

Lasso(alpha=0.0001, max_iter=10000000)

In [31]:
evaluate_performance(lasso_raw.predict(pk_vss_test[features]), pk_vss_test[target_vss])

----> RMSE= 		52.185708161339804
----> MAE= 		6.827111727723389
----> R^2= 		0.002310849805227666


In [33]:
evaluate_performance(lasso_raw.predict(pk_vss_test[features]), pk_vss_test[target_vss])

----> RMSE= 		52.185708161339804
----> MAE= 		6.827111727723389
----> R^2= 		0.002310849805227666


## looking at the feature importance

In [34]:
print(list(compress(features, lasso_raw.coef_ > 0.001)))

print(list(compress(features, lasso_raw.coef_ > 1)))

print(list(compress(features, lasso_raw.coef_ > 2)))

['MW', 'RotBondCount', 'MoKa.LogP', 'Chi0n', 'Chi1v', 'Chi2n', 'Chi3v', 'Chi4n', 'allKierAlpha', 'Kappa2', 'LabuteASA', 'N_AliphCarbC', 'N_AmideBonds', 'N_AromCarbC', 'N_AromRing', 'N_BridgeheadAtoms', 'N_HBA', 'N_Heteroatoms', 'N_LipinskiHBD', 'N_Rings', 'N_SatHetC', 'N_UnsAtSterCent']
['Chi0n', 'Chi1v', 'Chi2n', 'Chi3v', 'Chi4n', 'allKierAlpha', 'N_AliphCarbC', 'N_HBA', 'N_LipinskiHBD']
['Chi0n', 'Chi2n', 'allKierAlpha', 'N_AliphCarbC']


# 2. log(data)

In [ ]:
lasso_log = linear_model.Lasso(alpha=0.0001, max_iter=10_000_000)
lasso_log.fit(pk_vss_train[features], pk_vss_train[target_log_vss])

In [ ]:
evaluate_performance(lasso_log.predict(pk_vss_test[features]), pk_vss_test[target_log_vss])

## again, looking at the feature importance

In [ ]:
print(list(compress(features, lasso_log.coef_ > 0.001)))

print(list(compress(features, lasso_log.coef_ > 1)))

print(list(compress(features, lasso_log.coef_ > 2)))

# Evaluating ridge regression (Vss)

## GridSearch

# 1. raw data

In [ ]:
ridge_raw = Ridge()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# search space for the gridsearch
space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
space['fit_intercept'] = [True, False]
space['normalize'] = [True, False]


search = GridSearchCV(ridge_raw, space, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
result = search.fit(pk_vss_train[features], pk_vss_train[target_vss])

In [ ]:
# checking out the best parameters:

print(result.best_params_)

In [ ]:
# .. and using them for modelling
ridge_raw = Ridge(alpha= 0.1, fit_intercept=True, normalize= True, solver= 'svd')

In [ ]:
ridge_raw.fit(pk_vss_train[features], pk_vss_train[target_vss])

In [ ]:
evaluate_performance(ridge_raw.predict(pk_vss_test[features]), pk_vss_test[target_vss])

# 2. log(data)

In [ ]:
ridge_log = Ridge()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# search space for the gridsearch
space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
space['fit_intercept'] = [True, False]
space['normalize'] = [True, False]


search = GridSearchCV(ridge_log, space, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
result = search.fit(pk_vss_train[features], pk_vss_train[target_log_vss])

In [ ]:
# checking out the best parameters:

print(result.best_params_)

In [ ]:
# .. and using them for modelling
ridge_log = Ridge(alpha= 10, fit_intercept=False, normalize= True, solver= 'cholesky')

In [ ]:
ridge_log.fit(pk_vss_train[features], pk_vss_train[target_log_vss])

In [ ]:
evaluate_performance(ridge_log.predict(pk_vss_test[features]), pk_vss_test[target_log_vss])

# Evaluating ElasticNet (Vss)

## GridSearch

# 1. raw data

In [ ]:
elastic_net_raw = ElasticNet()

alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
l1_ratio = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
param_grid = dict(alpha=alpha, l1_ratio=l1_ratio)

grid = GridSearchCV(estimator=elastic_net_raw, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)

grid_result = grid.fit(pk_vss_train[features], pk_vss_train[target_vss])

In [ ]:
# checking best parameters
grid_result.best_params_

In [ ]:
# using them for modelling
elastic_net_raw = ElasticNet(alpha=.1, l1_ratio=.8, max_iter = 100_000).fit(pk_vss_train[features], pk_vss_train[target_vss])

In [ ]:
evaluate_performance(elastic_net_raw.predict(pk_vss_test[features]), pk_vss_test[target_vss])

# 2. log(data)

In [ ]:
elastic_net_log = ElasticNet()

alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
l1_ratio = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
param_grid = dict(alpha=alpha, l1_ratio=l1_ratio)

grid = GridSearchCV(estimator=elastic_net_log, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)

grid_result = grid.fit(pk_vss_train[features], pk_vss_train[target_log_vss])

In [ ]:
# checking best parameters
grid_result.best_params_

In [ ]:
# using them for modelling
elastic_net_log = ElasticNet(alpha=.1, l1_ratio=0, max_iter = 100_000).fit(pk_vss_train[features], pk_vss_train[target_log_vss])

In [ ]:
evaluate_performance(elastic_net_log.predict(pk_vss_test[features]), pk_vss_test[target_log_vss])

# Evaluating Random forest (Vss)

## GridSearch

# 1. raw data

In [ ]:
tuned_parameters = {'n_estimators': [500, 700, 1000], 'max_depth': [None, 1, 2, 3], 'min_samples_split': [1, 2, 3]}


rf_raw = GridSearchCV(RandomForestRegressor(), tuned_parameters, cv=5, 
                   n_jobs=-1, verbose=1)

rf_raw.fit(pk_vss_train[features], pk_vss_train[target_vss])

In [ ]:
# best parameters:

rf_raw.best_params_

In [ ]:
# using them for modelling:

rf_raw = RandomForestRegressor(max_depth= 2, min_samples_split= 3, n_estimators= 1000).fit(pk_vss_train[features], pk_vss_train[target_vss])

In [ ]:
evaluate_performance(rf_raw.predict(pk_vss_test[features]), pk_vss_test[target_vss])

# 2. log(data)

In [ ]:
tuned_parameters = {'n_estimators': [500, 700, 1000], 'max_depth': [None, 1, 2, 3], 'min_samples_split': [1, 2, 3]}


rf_log = GridSearchCV(RandomForestRegressor(), tuned_parameters, cv=5, 
                   n_jobs=-1, verbose=1)

rf_log.fit(pk_vss_train[features], pk_vss_train[target_log_vss])

In [ ]:
# best parameters:

rf_log.best_params_

In [ ]:
# using them for modelling:

rf_log = RandomForestRegressor(max_depth= None, min_samples_split= 3, n_estimators= 1000).fit(pk_vss_train[features], pk_vss_train[target_log_vss])

In [ ]:
evaluate_performance(rf_log.predict(pk_vss_test[features]), pk_vss_test[target_log_vss])